In [ ]:
'''
This project focuses on using public data, in order to identify possible cases of collusion or bid manipulation.
The main objective is to tackle corruption by harnessing technological tools, like the one that was developed
for this iniciative.
'''

import json
import re
import polars as pl
from dateutil.parser import parse
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create an empty list
data = []

# Upload files with widgets.FileUpload
upload_widget = widgets.FileUpload(aceptar='.json',multiple=True)

# Display the upload widget
display(upload_widget)

# Function for the json file
def handle_upload(change):
    global data
    uploaded_files = upload_widget.value

    for file_nombre, file_info in uploaded_files.items():
        # Process the file data
        file_content = file_info['content']

        # Load the file content as JSON
        try:
            json_data = json.loads(file_content)
            if isinstance(json_data, dict):
                json_data = [json_data]
        except json.JSONDecodeError:
            print(f"Error decoding JSON from file: {file_nombre}")
            continue

        # Take the state variable considering the name of the json file.
        #Change the format to get the state name correctly.
        state = file_nombre.split('.')[0].replace('_releases', '')
        state = re.sub(r'([a-z])([A-Z])', r'\1 \2', state).title()
        state = re.sub(r'\s\(\d+\)', '', state)

        # Process each JSON entry
        for entry in json_data:
            # # Extract necessary fields
            '''
            Variables:
            - Id of the contract
            - Description of the tender
            - Supplier
            - Value of the tender
            - Value of the contract
            - Period of the tender
            '''

            #Finds contract id, from json file
            contract_id = entry.get("_id", {}).get("$oid", None) if isinstance(entry.get("_id", {}), dict) else entry.get("_id", None)

            #Finds information within the tender variable
            tender_description = entry.get("tender", {}).get("description", "") or "No hay información"
            tender_value = entry.get("tender", {}).get("value", {}).get("amount", 0)

            #Finds the contracts and awards
            contracts = entry.get("contracts", [])
            awards = entry.get("awards", [])

            #Finds the name of the suppliers
            awards_supplier = ", ".join(
                [supplier.get("name", "") for award in awards for supplier in award.get("suppliers", [])]
            ) or "No hay información" #In some cases it doesn't have information for the name of the supplier.

            total_contract_amount = sum(contract.get("value", {}).get("amount", 0) for contract in contracts) #In case there are more contracts, it adds all the value amounts.

            #Gets the variable for the period of the tender
            period = entry.get("tender", {}).get("tenderPeriod", {})
            #Get the start and end date of the tender variable.
            start_date = period.get("startDate")
            end_date = period.get("endDate")


            # Calculate the duration in days from the publication til the deadline. If it's less than 15, then there's a red flag.
            try:
              #Parse is needed due to the date format of the variables. It is useful to read it.
                if start_date and end_date:
                    start_date_parsed = parse(start_date)
                    end_date_parsed = parse(end_date)
                    days_duration = (end_date_parsed - start_date_parsed).days
                    tenderPeriod_flag = '🔴' if days_duration < 15 else '🟢' #This parameter is according to the "Ley de Adquisiciones, Arrendamientos y Servicios del Sector Público"
                else:
                    days_duration = None # The date fields were optional, which means not all contracts will have it
                    tenderPeriod_flag = None
            except (ValueError, TypeError):
                print(f"Error parsing dates in file {file_nombre}")
                days_duration = None
                tenderPeriod_flag = None

            # Calculate the difference between the tender amount and the contract amount to identify possible collusion cases
            tender_percent_diff = (((total_contract_amount - tender_value) / tender_value) * 100 if tender_value else 0)
            red_flag_bid_tender = '🔴' if tender_percent_diff >= 30 else '🟢' #The 30% is considering the acceptable percentage for the World Bank.

            # Count the number of red flags
            red_flag_count = sum([1 for flag in [tenderPeriod_flag, red_flag_bid_tender] if flag == '🔴'])

            # Append to data
            data.append({
                "Id": contract_id,
                "Estado": state,
                "Valor tender": tender_value,
                "Cantidad de contrato": total_contract_amount,
                "Flag_tender": red_flag_bid_tender,
                "Descripción de contrato": tender_description,
                "Proveedores": awards_supplier,
                "Periodo de Tiempo (días)": days_duration if days_duration is not None else "No hay información",
                "TenderPeriod": tenderPeriod_flag if tenderPeriod_flag is not None else "No hay información",
                "Sum": red_flag_count
            })

    # Create a Polars DataFrame
    df = pl.DataFrame(data)

    # Creation of filters (flags, state, id, and suppliers).
    def filter_data(flag_type, state_filter, id_filter, supplier_filter):
        filtered_df = df
        if flag_type == 'Red Flags':
            filtered_df = filtered_df.filter(pl.col('Flag_tender') == '🔴')
        elif flag_type == 'Green Flags':
            filtered_df = filtered_df.filter((pl.col('Flag_tender') == '🟢') & (pl.col('TenderPeriod') != '🔴'))
        if state_filter:
            filtered_df = filtered_df.filter(pl.col('Estado').str.to_lowercase().str.contains(state_filter.lower()))
        if id_filter:
            filtered_df = filtered_df.filter(pl.col('Id').str.to_lowercase().str.contains(id_filter.lower()))
        if supplier_filter:
            filtered_df = filtered_df.filter(pl.col('Proveedores').str.to_lowercase().str.contains(supplier_filter.lower()))
        return filtered_df

    # Create Widgets to make it interactive
    flag_dropdown = widgets.Dropdown(options=['All Flags', 'Red Flags', 'Green Flags'],
                                     value='All Flags',
                                     description='Bandera:',
                                     layout=widgets.Layout(width='200px'))

    state_text = widgets.Text(
        value='',
        placeholder='Estado',
        description='Estado:',
        layout=widgets.Layout(width='300px')
    )


    id_text = widgets.Text(
        value='',
        placeholder='ID del contrato',
        description='ID:',
        layout=widgets.Layout(width='300px')
    )

    supplier_text = widgets.Text(
        value='',
        placeholder='Proveedor',
        description='Proveedor:',
        layout=widgets.Layout(width='300px')
    )

    # Counter for all red and green flags.
    red_flag_count = widgets.Label(value="🔴 Red Flags: 0")
    green_flag_count = widgets.Label(value="🟢 Green Flags: 0")

    # When the user uses the filters, the flag count will change accordingly
    def update_flag_counts(filtered_df):
      #Filters for only red or green flag
        red_count = filtered_df.filter(pl.col('TenderPeriod') == '🔴').shape[0]
        green_count = filtered_df.filter(pl.col('TenderPeriod') == '🟢').shape[0]
        # Counts the number of flags after the filter
        red_flag_count.value = f"🔴 Red Flags: {red_count}"
        green_flag_count.value = f"🟢 Green Flags: {green_count}"

    # Updated function
    output = widgets.Output()

    # Update the table to show results from the filters.
    def update_table(change=None):
        with output:
            clear_output(wait=True)
            filtered_df = filter_data(flag_dropdown.value, state_text.value, id_text.value, supplier_text.value)

            display(filtered_df.to_pandas().style.set_properties(**{'text-align': 'left'}))

            # Calculated the sum of all the red flags, so that each time the user filters something, the counter gets updated.
            red_flags = (filtered_df['Flag_tender'] == '🔴').sum() + (filtered_df['TenderPeriod'] == '🔴').sum() #The addition is to count all the flags.
            green_flags = (filtered_df['Flag_tender'] == '🟢').sum() + (filtered_df['TenderPeriod'] == '🟢').sum()
            red_flag_count.value = f"🔴 Red Flags: {red_flags}"
            green_flag_count.value = f"🟢 Green Flags: {green_flags}"

    # It changes the widget when filtered
    flag_dropdown.observe(update_table, names='value')
    state_text.observe(update_table, names='value')
    id_text.observe(update_table, names='value')
    supplier_text.observe(update_table, names='value')

    # Display widgets and output
    display(widgets.VBox([
        widgets.HBox([red_flag_count, green_flag_count]),
        widgets.HBox([flag_dropdown, state_text, id_text, supplier_text]),
        output
    ]))

    # Once it has updated according to the filters, the results will be desplayed in the table.
    update_table()

# Attach the function to the file upload widget
upload_widget.observe(handle_upload, names='value')